# Проверка сервиса ASR

In [1]:
# Путь к тестовому WAV-файлу
audio_file_path = "C:\\_tmp\\E1.wav"

In [2]:
import requests
url = "http://localhost:8200/process-audio/"

# Путь к тестовому WAV-файлу
#audio_file_path = "test_audio.wav"

# Путь к тестовому WAV-файлу
#audio_file_path = "C:\\_tmp\\E4.wav"

# Список тестовых фраз
#phrases = "hello world;test phrase;how are you;Открыть файл"

# Отправка POST-запроса
with open(audio_file_path, "rb") as audio_file:
    response = requests.post(
        url,
        files={"file": ("test_audio.wav", audio_file, "audio/wav")},
#        data={"phrases": phrases}
    )

# Вывод результата
if response.status_code == 200:
    print("Response JSON:", response.json())
else:
    print(f"Error: {response.status_code}", response.text)


Response JSON: {'recognized_text': 'доброе утро дежурная бригада начинаем утренний диписи испытатель е один дата восемнадцатое апреля'}


In [1]:
import torch
#Определим устройство для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
from transformers import pipeline
from scipy.io import wavfile
import numpy as np
#транскрибатор
transcriber = pipeline("automatic-speech-recognition", model="lorenzoncina/whisper-medium-ru")




samplerate, data = wavfile.read(audio_file_path)

if data.ndim > 1:
    data = data.mean(axis=1)
    
data = data.astype(np.float32)
data /= np.max(np.abs(data))
question = transcriber({"sampling_rate": samplerate, "raw": data})["text"]  
question


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
c:\Users\rvv19\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


'Добрый вечер, уважаемая дежурная бригада, испытатель ей четыре, сегодня двадцать первое апреля двадцать один сорок девять, настроение меня отличное, самочуствие тупое.'

In [7]:
samplerate

44100

In [ ]:
transcriber.model.sav

TypeError: Pipeline.check_model_type() missing 1 required positional argument: 'supported_models'

In [41]:
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment

FRAME_RATE = 16000 # Нужная частота дискретизации
CHANNELS = 1 # Нужное количество каналов


# Модель из папки которую скачали ранее
model = Model('vosk-model-ru-0.22')
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

In [52]:
mp3 = AudioSegment.from_wav(audio_file_path)#  from_mp3(os.path.join(FOLDER_MP3, file_mp3))
mp3 = mp3.set_channels(CHANNELS)
mp3 = mp3.set_frame_rate(FRAME_RATE)

rec.AcceptWaveform(mp3.raw_data)
result = rec.Result()

In [31]:
mp3.raw_data

b'\x0c\x00#\x00\xf6\xff\xa9\xff\x81\xffZ\xff\x85\xff\xe7\xffX\x00g\x00T\x003\x00\'\x00\x07\x00"\x00:\x00a\x00t\x00_\x00o\x00@\x00\xed\xff\xca\xff\xe8\xff\x1f\x00\xfd\xff\xcc\xff\xd6\xff\xeb\xff\xc8\xff|\xffW\xff\x95\xff\xc2\xff\x12\x004\x00\x11\x00\xe8\xff\xc4\xff\xf0\xff1\x00)\x00>\x00O\x00m\x00\x84\x00f\x001\x00\xf8\xff\xb9\xff\x84\xff4\xff\xd7\xfe\xd3\xfe\xaa\xfe\xc9\xfe\xec\xfe\x00\xff\xcc\xfe\xed\xfe\x00\xff\x14\xff~\xff\xc5\xff\x9e\xff\xb8\xff\xf8\xff(\x00S\x00\x8e\x00\xc1\x00\xa4\x00\x7f\x00S\x00=\x00o\x00E\x00\'\x00\'\x00\xeb\xff\xa1\xffg\xff\x81\xff{\xff\xa4\xff\xb9\xff\xdc\xff\x00\x00*\x00\x81\x00\xc3\x00\xc8\x00\xeb\x00&\x01\x06\x01\xff\x00\xcc\x00\xcb\x00\xbb\x00\xa9\x00\xe1\x00\xb0\x00a\x00(\x00\xee\xff\xa0\xff\x83\xffT\xff\xaf\xfe\xf6\xfd\x86\xfdI\xfd8\xfdp\xfd\x90\xfd\xb1\xfd\x8c\xfdx\xfdw\xfd\xc0\xfd\xd3\xfd\xe9\xfd\x13\xfeb\xfe\x99\xfef\xfeB\xfeb\xfeF\xfeV\xfe\x91\xfe\xd6\xfe\x02\xff/\xffF\xff*\xff\xd9\xfe\xa4\xfe^\xfe+\xfeJ\xfe?\xfe\xfb\xfd\xe2\xfd\xd4\xfd\xae\xfd\xaa

In [44]:
import librosa
y, s = librosa.load(audio_file_path, sr=FRAME_RATE) # Downsample 44.1kHz to 8kHz

In [45]:
# Модель из папки которую скачали ранее
model = Model('vosk-model-ru-0.22')
rec = KaldiRecognizer(model, s )
rec.SetWords(True)

In [46]:
rec.AcceptWaveform(y.tobytes())
result = rec.Result()
json.loads(result)["text"]

''

In [53]:
import json
json.loads(result)["text"]

'настроение у меня отличное самочувствие добрый вечер уважаемая дежурная бригада испытателей четыре сегодня двадцать первое апреля двадцать один сорок девять'

# Проверка сервиса TTS

In [5]:
# URL сервиса FastAPI
port = '8080'
speaker = 'baya'  # aidar, baya, kseniya, xenia, eugene
words = "прием прием начинаем ежедневную планерку"
sample_rate = 8000# 8000, 24000, 48000
# URL сервиса FastAPI

url = f"http://localhost:{port}/get_audio_file/?words=\"{words}\"&speaker={speaker}&sample_rate={sample_rate}"
response = requests.get(url)
# Проверяем, успешно ли прошел запрос
if response.status_code == 200:
    # Сохраняем полученный файл
    with open("processed_sound.wav", "wb") as output_sound:
        output_sound.write(response.content)
    print("Сгенерированный звук сохранен как processed_sound.wav")
else:
    print("Ошибка при отправке запроса:", response.status_code)


Сгенерированный звук сохранен как processed_sound.wav


In [6]:
import IPython
IPython.display.Audio("processed_sound.wav")

In [9]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:8000/v1',
    api_key='not-needed',
)

history = [
    {
        "role": "system",
        "content": "You are an intelligent assistant."
    }
]

history.append({"role": "user", "content": "столица россии?"})
completion = client.chat.completions.create(
        model="local-model",
        messages=history,
        temperature=0.7,
    )
content = completion.choices[0].message.content

In [10]:
content

'Москва.'

In [3]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import requests

class CustomLLM:
    def __init__(self, endpoint: str):
        self.endpoint = endpoint

    def __call__(self, prompt: str) -> str:
        response = requests.post(
            url=self.endpoint,
            json={"prompt": prompt, "max_tokens": 512}  # Замените параметрами вашего API
        )
        if response.status_code == 200:
            return response.json().get("generated_text", "")
        else:
            raise Exception(f"Error {response.status_code}: {response.text}")

# Указываем URL вашего сервера
server_url = "http://localhost:8000"  # Замените на нужный адрес

# Создаем кастомный LLM
llm = CustomLLM(endpoint=f"{server_url}/generate")  # Уточните путь API

# Определяем шаблон для запроса
template = "You are a helpful assistant. Answer this: {question}"
prompt = PromptTemplate(input_variables=["question"], template=template)

# Интеграция LLM с LangChain
chain = LLMChain(llm=llm, prompt=prompt)

# Используем chain
question = "What is the capital of France?"
response = chain.run(question=question)

print(response)

C:\Users\rvv19\AppData\Local\Temp\ipykernel_116524\968737201.py:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<__main__.CustomLLM object at 0x000002324A8866C0>, input_type=CustomLLM]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<__main__.CustomLLM object at 0x000002324A8866C0>, input_type=CustomLLM]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [2]:
from langchain.prompts import PromptTemplate
from langchain.schema import Runnable
from langchain.chains import RunnableSequence
import requests

class CustomLLM:
    def __init__(self, endpoint: str):
        self.endpoint = endpoint

    def __call__(self, prompt: str) -> str:
        response = requests.post(
            url=self.endpoint,
            json={"prompt": prompt, "max_tokens": 512}  # Настройте параметры API
        )
        if response.status_code == 200:
            return response.json().get("generated_text", "")
        else:
            raise Exception(f"Error {response.status_code}: {response.text}")

# Указываем URL вашего сервера
server_url = "http://localhost:8000"  # Замените на нужный адрес

# Создаем кастомный LLM
llm = CustomLLM(endpoint=f"{server_url}/generate")  # Уточните путь API

# Определяем шаблон для запроса
template = "You are a helpful assistant. Answer this: {question}"
prompt = PromptTemplate(input_variables=["question"], template=template)

# Интеграция с новым подходом через RunnableSequence
chain = RunnableSequence(
    steps=[
        prompt,
        Runnable(llm)  # Оборачиваем LLM для использования в цепочке
    ]
)

# Используем chain
question = "What is the capital of France?"
response = chain.invoke({"question": question})

print(response)


ImportError: cannot import name 'Runnable' from 'langchain.schema' (c:\Users\rvv19\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\schema\__init__.py)

In [1]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

# Define the model path
model_path = "/path/to/your/model/openorca-platypus2-13b.gguf.q4_0.bin"
# Указываем URL вашего сервера
server_url = "http://localhost:8000"  # Замените на нужный адрес
# Set up callback manager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Create the LLM object
llm = LlamaCpp(
    model_path=server_url, 
    temperature=0.7,
    max_tokens=200,
    top_p=1.0,
    callback_manager=callback_manager,
    verbose=True
)

# Example usage
question = "What is bindings in programming languages?"
response = llm.invoke({"text" : question})
print(response)

ValidationError: 1 validation error for LlamaCpp
  Value error, Could not load Llama model from path: http://localhost:8000. Received error Model path does not exist: http://localhost:8000 [type=value_error, input_value={'model_path': 'http://lo...: None, 'grammar': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [7]:
import time
time.asctime(time.gmtime())

'Sun Jan 26 18:29:25 2025'

In [10]:
from time import gmtime, strftime
strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())

'Sun_26_Jan_2025_18_50_24'

: 

In [3]:
import os
import torch

device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model.pt'

if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v4_ru.pt',
                                   local_file)  

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)

example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра кедров.'
sample_rate = 48000
speaker='baya'

audio_paths = model.save_wav(text=example_text,
                             speaker=speaker,
                             sample_rate=sample_rate)

In [4]:
audio_paths

'test.wav'